# PEFT 进阶操作

## 1. 自定义模型适配

In [1]:
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net1 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [3]:
for name, param in net1.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias


In [4]:
config = LoraConfig(target_modules=["0"])

In [5]:
model1 = get_peft_model(net1, config)

In [6]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

## 2. 多适配器加载与切换

In [7]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [8]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2, config1)
model2.save_pretrained("./loraA")

In [9]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

In [ ]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

In [ ]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

In [ ]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

In [ ]:
model2.active_adapter

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
for name, param in model2.named_parameters():
    print(name, param)

In [ ]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
model2.set_adapter("loraB")

In [ ]:
model2.active_adapter

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

## 3. 禁用适配器

In [ ]:
model2.set_adapter("loraA")

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))